# Análise de Dados de Infrações de Trânsito - 2019
### Fonte: http://www.dados.df.gov.br/dataset/infracoes-transito

In [1]:
# Importando as bibliotecas do Python

import pandas as pd
from pandasql import sqldf
import numpy as np
from datetime import datetime
from itertools import product
import os
import glob
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
# Visualizando os arquivos
folder_name = 'C:\\PROJETOS_GIT_DEV\\Infracoes_Transito_2019\\Infracoes2019'
file_type = 'csv'
for f in glob.glob(folder_name + "/*."+file_type):
    print(f)

C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\agosto.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\derdadosabertosabr2019.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\derdadosabertosfev2019.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\derdadosabertosjan2019.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\derdadosabertosjunho2019.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\derdadosabertosmai2019-2.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\derdadosabertosmar2019.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\julho.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\outubro.csv
C:\PROJETOS_GIT_DEV\Infracoes_Transito_2019\Infracoes2019\setembro.csv


In [3]:
# Lendo os arquivos. Convertendo o dataset em um dataframe com Pandas
seperator =';'
df = pd.concat([pd.read_csv(f, sep=seperator) for f in glob.glob(folder_name + "/*."+file_type)],ignore_index=True)

In [4]:
# Quantidade de linhas
print('Quantidade de registros:', len(df.index))
#df.describe()

Quantidade de registros: 757457


In [5]:
# Visualizando os dados
#print(df.keys())
df.head(3)

,tipo_infracao,descricao,tipo_infrator,tipo_veiculo,cometimento,hora_cometimento,auinf_local_rodovia,auinf_local_km,auinf_local_referencia,auinf_local_complemento,auinf_local_latitude,auinf_local_longitude,grav_tipo
0,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,AUTOMOVEL,01/08/2019,00:08,V. DE LIGACAO DF-001(EPCT)/DF-075(EPNB) VIADUT...,NaN,NaN,NaN,NaN,NaN,Grave
1,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,AUTOMOVEL,01/08/2019,00:10,"DF-075 (EPNB) KM 0,5 SENTIDO SAMAMBAIA ...",NaN,NaN,NaN,NaN,NaN,Média
2,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,AUTOMOVEL,01/08/2019,00:10,"DF-075 (EPNB) KM 5,7 SENTIDO DF-003 (EPIA) ...",NaN,NaN,NaN,NaN,NaN,Grave


In [6]:
# Verificando tipo dos atributos
df.dtypes

tipo_infracao              object
descricao                  object
tipo_infrator              object
tipo_veiculo               object
cometimento                object
hora_cometimento           object
auinf_local_rodovia        object
auinf_local_km             object
auinf_local_referencia     object
auinf_local_complemento    object
auinf_local_latitude       object
auinf_local_longitude      object
grav_tipo                  object
dtype: object

In [7]:
# Verificando a quantidade de valores nulos nas colunas
df.isnull().sum()

tipo_infracao                   0
descricao                       0
tipo_infrator                   0
tipo_veiculo                 8137
cometimento                     0
hora_cometimento                0
auinf_local_rodovia           113
auinf_local_km             578541
auinf_local_referencia     566444
auinf_local_complemento    726788
auinf_local_latitude       578007
auinf_local_longitude      578007
grav_tipo                       0
dtype: int64

In [8]:
# Selecionando as colunas sem valores nulos
df = pd.DataFrame(df,columns=['tipo_infracao','descricao','tipo_infrator','cometimento','hora_cometimento','grav_tipo'])

In [9]:
# Função para definir o id da gravidade da infração
def define_id_gravidade(grav):
    if grav == 'Leve':
        id_grav = 1
    elif grav == 'Média':
        id_grav = 2
    elif grav == 'Grave':
        id_grav = 3
    elif grav == 'Gravíssima':
        id_grav = 4
    return id_grav

In [10]:
df['id_grav_tipo'] = df.grav_tipo.apply(define_id_gravidade)

In [11]:
# Concatenando as colunas 'cometimento' e 'hora_cometimento'
df['data_hora_cometimento'] = df['cometimento'] + ' ' + df['hora_cometimento']

In [12]:
df.head()

,tipo_infracao,descricao,tipo_infrator,cometimento,hora_cometimento,grav_tipo,id_grav_tipo,data_hora_cometimento
0,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:08,Grave,3,01/08/2019 00:08
1,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:10,Média,2,01/08/2019 00:10
2,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:10,Grave,3,01/08/2019 00:10
3,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:11,Grave,3,01/08/2019 00:11
4,7471-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:12,Gravíssima,4,01/08/2019 00:12


In [13]:
# Convertendo a coluna 'data_hora_cometimento' em datetime
df['data_hora_cometimento'] = df['data_hora_cometimento'].astype('datetime64')

In [14]:
df.head()

,tipo_infracao,descricao,tipo_infrator,cometimento,hora_cometimento,grav_tipo,id_grav_tipo,data_hora_cometimento
0,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:08,Grave,3,2019-01-08 00:08:00
1,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:10,Média,2,2019-01-08 00:10:00
2,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:10,Grave,3,2019-01-08 00:10:00
3,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:11,Grave,3,2019-01-08 00:11:00
4,7471-0,Transitar em velocidade superior à máxima perm...,Condutor,01/08/2019,00:12,Gravíssima,4,2019-01-08 00:12:00


In [15]:
# Excluindo as colunas 'cometimento' e 'hora_cometimento'
del df['cometimento']
del df['hora_cometimento']

In [16]:
df.head()

,tipo_infracao,descricao,tipo_infrator,grav_tipo,id_grav_tipo,data_hora_cometimento
0,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:08:00
1,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,Média,2,2019-01-08 00:10:00
2,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:10:00
3,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:11:00
4,7471-0,Transitar em velocidade superior à máxima perm...,Condutor,Gravíssima,4,2019-01-08 00:12:00


In [17]:
# Definindo o id do dia da semana
df['id_dia_semana'] = df['data_hora_cometimento'].dt.weekday    

In [18]:
# Definindo o nome do dia da semana
df['nome_dia_semana'] = df.data_hora_cometimento.dt.day_name(locale='pt')

In [19]:
# Definindo o id do mês
df['id_mes'] = df['data_hora_cometimento'].dt.month

In [20]:
# Definindo o nome do mês
df['nome_mes'] = df.data_hora_cometimento.dt.month_name(locale = 'Portuguese') 

In [21]:
df.head()

,tipo_infracao,descricao,tipo_infrator,grav_tipo,id_grav_tipo,data_hora_cometimento,id_dia_semana,nome_dia_semana,id_mes,nome_mes
0,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:08:00,1,Terça-feira,1,Janeiro
1,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,Média,2,2019-01-08 00:10:00,1,Terça-feira,1,Janeiro
2,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:10:00,1,Terça-feira,1,Janeiro
3,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:11:00,1,Terça-feira,1,Janeiro
4,7471-0,Transitar em velocidade superior à máxima perm...,Condutor,Gravíssima,4,2019-01-08 00:12:00,1,Terça-feira,1,Janeiro


In [22]:
# Função para definir o período do dia
def define_nome_periodo(hora):
    madrugada = [0, 1, 2, 3, 4, 5]
    manha = [6, 7, 8, 9, 10, 11, 12]
    tarde = [13, 14, 15, 16, 17, 18]
    noite = [19, 20, 21, 22, 23]
    if hora in madrugada:
        nome_periodo = 'Madrugada'
    elif hora in manha:
        nome_periodo = 'Manhã'
    elif hora in tarde:
        nome_periodo = 'Tarde'
    elif hora in noite:
        nome_periodo = 'Noite'
    return nome_periodo

In [23]:
# Função para definir o id do período do dia
def define_id_periodo(nome_periodo):
    if nome_periodo == 'Madrugada':
        id_periodo = 1
    elif nome_periodo == 'Manhã':
        id_periodo = 2
    elif nome_periodo == 'Tarde':
        id_periodo = 3
    elif nome_periodo == 'Noite':
        id_periodo = 4
    return id_periodo

In [24]:
df['nome_periodo'] = df.data_hora_cometimento.dt.hour.apply(define_nome_periodo)

In [25]:
df['id_periodo'] = df.nome_periodo.apply(define_id_periodo)

In [26]:
df.head()

,tipo_infracao,descricao,tipo_infrator,grav_tipo,id_grav_tipo,data_hora_cometimento,id_dia_semana,nome_dia_semana,id_mes,nome_mes,nome_periodo,id_periodo
0,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:08:00,1,Terça-feira,1,Janeiro,Madrugada,1
1,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,Média,2,2019-01-08 00:10:00,1,Terça-feira,1,Janeiro,Madrugada,1
2,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:10:00,1,Terça-feira,1,Janeiro,Madrugada,1
3,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:11:00,1,Terça-feira,1,Janeiro,Madrugada,1
4,7471-0,Transitar em velocidade superior à máxima perm...,Condutor,Gravíssima,4,2019-01-08 00:12:00,1,Terça-feira,1,Janeiro,Madrugada,1


In [27]:
# Verificando o id_dia_semana
q = """SELECT DISTINCT id_dia_semana, nome_dia_semana FROM df;"""
pysqldf = lambda q: sqldf(q, globals())
query_df = pysqldf(q)

In [28]:
query_df

,id_dia_semana,nome_dia_semana
0,1,Terça-feira
1,4,Sexta-feira
2,0,Segunda-feira
3,2,Quarta-feira
4,5,Sábado
5,3,Quinta-feira
6,6,Domingo


In [29]:
# Funçao para ajustar o id do dia da semana
def ajusta_id_dia_semana(id_dia):
    if id_dia == 6: #domingo
        novo_id = 1
    elif id_dia == 0: #segunda
        novo_id = 2
    elif id_dia == 1: #terça
        novo_id = 3
    elif id_dia == 2: #quarta
        novo_id = 4
    elif id_dia == 3: #quinta
        novo_id = 5
    elif id_dia == 4: #sexta
        novo_id = 6
    elif id_dia == 5: #sábado
        novo_id = 7
    return novo_id

In [30]:
# Ajustando os valores da coluna id_dia_semana
df['id_dia_semana'] = df.id_dia_semana.apply(ajusta_id_dia_semana)

In [31]:
df.head()

,tipo_infracao,descricao,tipo_infrator,grav_tipo,id_grav_tipo,data_hora_cometimento,id_dia_semana,nome_dia_semana,id_mes,nome_mes,nome_periodo,id_periodo
0,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:08:00,3,Terça-feira,1,Janeiro,Madrugada,1
1,7455-0,Transitar em velocidade superior à máxima perm...,Condutor,Média,2,2019-01-08 00:10:00,3,Terça-feira,1,Janeiro,Madrugada,1
2,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:10:00,3,Terça-feira,1,Janeiro,Madrugada,1
3,7463-0,Transitar em velocidade superior à máxima perm...,Condutor,Grave,3,2019-01-08 00:11:00,3,Terça-feira,1,Janeiro,Madrugada,1
4,7471-0,Transitar em velocidade superior à máxima perm...,Condutor,Gravíssima,4,2019-01-08 00:12:00,3,Terça-feira,1,Janeiro,Madrugada,1


In [32]:
# Verificando novamente o id_dia_semana
q2 = """SELECT DISTINCT id_dia_semana, nome_dia_semana FROM df;"""
pysqldf2 = lambda q2: sqldf(q2, globals())
query_df2 = pysqldf2(q2)

In [33]:
query_df2

,id_dia_semana,nome_dia_semana
0,3,Terça-feira
1,6,Sexta-feira
2,2,Segunda-feira
3,4,Quarta-feira
4,7,Sábado
5,5,Quinta-feira
6,1,Domingo


In [35]:
# Criando um dataframe com registros únicos de infração
df_infracao = pd.DataFrame(list(product(np.unique(df.tipo_infracao))), columns=['tipo_infracao'])

In [36]:
df_infracao.head()

,tipo_infracao
0,5010-0
1,5029-1
2,5029-2
3,5037-1
4,5037-2


In [37]:
# Inserindo ID ao dataframe
df_infracao.insert(0, 'id_tipo_infracao', range(1, 1 + len(df_infracao)))
df_infracao

,id_tipo_infracao,tipo_infracao
0,1,5010-0
1,2,5029-1
2,3,5029-2
3,4,5037-1
4,5,5037-2
5,6,5045-0
6,7,5053-1
7,8,5061-0
8,9,5070-2
9,10,5088-1


In [38]:
# Merge
df2 = pd.merge(df_infracao[['id_tipo_infracao','tipo_infracao']],df[['tipo_infracao', 'descricao', 'tipo_infrator', 'grav_tipo', 'id_grav_tipo', 'data_hora_cometimento', 'id_dia_semana', 'nome_dia_semana', 'id_mes', 'nome_mes', 'nome_periodo', 'id_periodo']],on='tipo_infracao', how='outer')

In [39]:
df2.head()

,id_tipo_infracao,tipo_infracao,descricao,tipo_infrator,grav_tipo,id_grav_tipo,data_hora_cometimento,id_dia_semana,nome_dia_semana,id_mes,nome_mes,nome_periodo,id_periodo
0,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 09:30:00,3,Terça-feira,1,Janeiro,Manhã,2
1,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 09:35:00,3,Terça-feira,1,Janeiro,Manhã,2
2,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 10:50:00,3,Terça-feira,1,Janeiro,Manhã,2
3,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 23:30:00,3,Terça-feira,1,Janeiro,Noite,4
4,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 23:55:00,3,Terça-feira,1,Janeiro,Noite,4


# Gerando CSV

In [40]:
# Alterando o índice da tabela para começar de 1
df2.index = [x for x in range(1, len(df2.values)+1)]
df2

,id_tipo_infracao,tipo_infracao,descricao,tipo_infrator,grav_tipo,id_grav_tipo,data_hora_cometimento,id_dia_semana,nome_dia_semana,id_mes,nome_mes,nome_periodo,id_periodo
1,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 09:30:00,3,Terça-feira,1,Janeiro,Manhã,2
2,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 09:35:00,3,Terça-feira,1,Janeiro,Manhã,2
3,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 10:50:00,3,Terça-feira,1,Janeiro,Manhã,2
4,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 23:30:00,3,Terça-feira,1,Janeiro,Noite,4
5,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 23:55:00,3,Terça-feira,1,Janeiro,Noite,4
6,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 00:10:00,6,Sexta-feira,2,Fevereiro,Madrugada,1
7,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 01:01:00,6,Sexta-feira,2,Fevereiro,Madrugada,1
8,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 01:15:00,6,Sexta-feira,2,Fevereiro,Madrugada,1
9,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 02:20:00,6,Sexta-feira,2,Fevereiro,Madrugada,1
10,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 13:33:00,6,Sexta-feira,2,Fevereiro,Tarde,3


In [41]:
# Nomeando a coluna do índice
df2.index.name = 'id_infracao'
df2

,id_tipo_infracao,tipo_infracao,descricao,tipo_infrator,grav_tipo,id_grav_tipo,data_hora_cometimento,id_dia_semana,nome_dia_semana,id_mes,nome_mes,nome_periodo,id_periodo
id_infracao,,,,,,,,,,,,,
1,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 09:30:00,3,Terça-feira,1,Janeiro,Manhã,2
2,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 09:35:00,3,Terça-feira,1,Janeiro,Manhã,2
3,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 10:50:00,3,Terça-feira,1,Janeiro,Manhã,2
4,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 23:30:00,3,Terça-feira,1,Janeiro,Noite,4
5,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-01-08 23:55:00,3,Terça-feira,1,Janeiro,Noite,4
6,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 00:10:00,6,Sexta-feira,2,Fevereiro,Madrugada,1
7,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 01:01:00,6,Sexta-feira,2,Fevereiro,Madrugada,1
8,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 01:15:00,6,Sexta-feira,2,Fevereiro,Madrugada,1
9,1,5010-0,Dirigir veículo sem possuir CNH/PPD/ACC,Condutor,Gravíssima,4,2019-02-08 02:20:00,6,Sexta-feira,2,Fevereiro,Madrugada,1


In [42]:
# Gravando csv para mostrar no Power BI
df2.to_csv('Infracoes_Transito_2019.csv')